# Jan 2019 New functionalities examples

Example demonstrating how to use the torque_tube functionality.

#### Prerequisites (Step 0):
We assume all instalation is working at this point, reffer to the other journals for this! 

# Torque Tube Example (Hex!)

In [1]:
import sys

sys.path.insert(0, '../bifacial_radiance')

try:
    from bifacial_radiance import *
    print " Successful import of bifacial_radiance version . "
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.

 Successful import of bifacial_radiance version . 


In [2]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp

timestamp = 4020 # Noon, June 17th. 
simulationname = 'Torque_tube_hex_test'

# MakeModule Parameters
module_type='my_custom_panel'
module_height = 1.996  # 2-up portrait Longi with 15cm additional gap
module_width = 0.991
orientation='portrait' 
tilt = 10

# TorqueTube Parameters
torqueTube = True
numpanels = 2
gap = 0.15
tubetype = 'Hex'
disttopanel = 0.175
diameter = 0.15
torqueTubeMaterial = 'Metal_Grey'       # IT's NOT GRAY, IT's GREY

# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 2.35   
azimuth_ang=90
sensorsy = 200 # Notice we will be doing 200 sensors! It takes more time but gives better resolution.
    

In [3]:
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\asdf2'

In [4]:
# Now let's run the example

demo = RadianceObj('Torque_tube_hex_test',path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit2(metdata,4020)  # Noon, June 17th

#We have a 2-up configuration in portrait. This section helps sort what is the COLLECTOR WIDTH to use.
if orientation == 'portrait':
                slope=module_height
elif orientation == 'landscape':  
                slope=module_width
collectorwidth = slope*numpanels+gap*(numpanels-1) # If it's 1UP, the gap value will not matter. This is a safety.

# Making module with all the variables
demo.makeModule(name=module_type,x=module_height,y=module_width,bifi=1,orientation='portrait', 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, material = torqueTubeMaterial, tubeZgap = disttopanel, numpanels = numpanels, panelgap = gap, rewriteModulefile = True)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': np.round(collectorwidth / gcr,3),'height':clearance_height,'orientation':orientation,'azimuth':azimuth_ang, 'module_type':module_type}  

scene = demo.makeScene(moduletype=module_type, sceneDict=sceneDict, nMods = 20, nRows = 7, sensorsy = sensorsy) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


path = C:\Users\sayala\Documents\RadianceScenes\asdf2
Making path: images/
Making path: objects/
Making path: results/
Making path: skies/
Making path: EPWs/
Making path: materials/
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
Sky generated with Gendaylit 2, with DNI: 242.0, DHI: 377.0
Datetime TimeIndex 1963-06-17 13:00:00-05:00

Module Name: my_custom_panel
Module file did not exist before, creating new module file
Module my_custom_panel successfully created
Created Torque_tube_hex_test.oct


At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:
    
   ## rvu -vf views\front.vp -e .01 Torque_tube_hex_test.oct
   
   And then proceed happily with your analysis:

In [ ]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
analysis.analysis(octfile, demo.name, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )


And then you dance-dance-dance. Except that, at the current stage of the software, if you open the results you will notice that now some of the sensors are measuring the irradinace at the torque tube and must die... in a future version.

(or increase the number of sensors by a lot and take them down in a separate routine in matlab all pretily and interpolate to have a consistent number of sensors with data while single axis tracking).

# Single-axis-tracking by HOUR example

Think of this as a fixed-tilt where the clearance height and the tracker tilt is changing. So the angles have to be re-calculated for every timestamp.

In [2]:
#Some variables again. Assuming bifacial_radiance was imported.

timeindex = 4020 # Noon, June 17th. 
simulationname = '1axisTrack'+str(timeindex)

# MakeModule Parameters
module_type='my_little_pony_panel'
module_height = 1.996  # 2-up portrait Longi with 15cm additional gap
module_width = 0.991
orientation='portrait' 
tilt = 10

# Tracking Angle Calculation Parameters
hub_height = 2.35   
axis_azimuth = 180.0
roundTrackerAngleBool = True 
angledelta = 5
limit_angle = 45
backtrack = True
    
# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo

In [3]:
#separate folder for this example
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\asdf2'

In [5]:
# Running the simulation
demo2 = RadianceObj(simulationname, path = testfolder)  # Create a RadianceObj 'object' named 'demo'
demo2.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo2.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo2.readEPW(epwfile) # read in the weather data
demo2.gendaylit2(metdata, 4020)  # Noon, June 17th

demo2.makeModule(name=module_type,x=module_width,y=module_height)    
tracker_theta, tracker_height, azimuth_ang = demo2._getTrackingGeometryTimeIndex(metdata, timeindex=timeindex, angledelta = angledelta, roundTrackerAngleBool = roundTrackerAngleBool, axis_azimuth = axis_azimuth, limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, hubheight = hub_height, module_height = module_height )
sceneDict = {'tilt':tracker_theta,'pitch': module_height / gcr,'height':tracker_height,'orientation':orientation,'azimuth':azimuth_ang, 'module_type':module_type}  
scene = demo2.makeScene(module_type, sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo2.makeOct(demo2.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.

path = C:\Users\sayala\Documents\RadianceScenes\asdf2
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
Sky generated with Gendaylit 2, with DNI: 242.0, DHI: 377.0
Datetime TimeIndex 1963-06-17 13:00:00-05:00

Module Name: my_little_pony_panel
Module file did not exist before, creating new module file
Module my_little_pony_panel successfully created
For this timestamp, panels are facing West
Tracker theta has been calculated to 4.722 and rounded to nearest tracking angle 5.0
Module clearance height has been calculated to 2.263, for this tracker theta.
The Oconv filelist is:
['materials\\ground.rad', 'skies\\sky2_1axisTrack4020.rad', 'objects\\my_little_pony_panel_2.263_6.048_20x7.rad']
Created 1axisTrack4020.oct


At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:
    
   ## rvu -vf views\front.vp -e .01 Torque_tube_hex_test.oct
   
   And then proceed happily with your analysis:

In [6]:
analysis = AnalysisObj(octfile, demo2.basename)  # return an analysis object including the scan dimensions for back irradiance
frontDict, backDict = analysis.analysis(octfile, demo2.basename, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio: %0.3f - %0.3f' %(min(analysis.backRatio), np.mean(analysis.backRatio)) )

Linescan in process: 1axisTrack4020_Front
Linescan in process: 1axisTrack4020_Back
Saved: results\irr_1axisTrack4020.csv
Annual bifacial ratio: 0.172 - 0.175


After you set up your system parameters, you can just change the timestamp and evaluate any day. You could also use gendaylit2manual to pass specific DNI/DHI if you so wanted.

Enjoy!